#  What Does Gradio Do?

One of the best ways to share your machine learning model, API, or data science workflow with others is to create an interactive app that allows your users or colleagues to try out the demo in their browsers.

# Learn to use Gradio

### 1.Install Gradio

In [29]:
pip install gradio

### 2.import gradio

In [30]:
import gradio as gr

### 3.Define your functional function

Here we have the simplest "Hello, World" as an example.

In [31]:
def greet(name):
    return "Hello " + name + "!"

### 4.Use the Interface Class to create a user interface wrapping our functional functions

The core Interface class is initialized with three required parameters:

fn: the function to wrap a UI around

inputs: which component(s) to use for the input (e.g. "text", "image" or "audio")

outputs: which component(s) to use for the output (e.g. "text", "image" or "label")

In [32]:
demo = gr.Interface(fn=greet, inputs="text", outputs="text")

### 5.launch result

After launching the demo, a link will be output about the relevant user interface.

In [33]:
demo.launch()

Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


If you want others to use this interface, just set 'share=True' in 'launch()'.

Then send the public URL to your users for them to use.

Please note: 

1.Because the processing happens on your device，your device should stay on!

2.These links are publicly accessible, meaning that anyone can use your model for prediction! Therefore, make sure not to expose any sensitive information through the functions you write, or allow any critical changes to occur on your device.

3.If you set share=False (the default, except in colab notebooks), only a local link is created.

4.This share link expires in 72 hours.

In [34]:
demo.launch(share=True)

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on local URL:  http://127.0.0.1:7881
Running on public URL: https://f24f4a0c9b714676.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


# Flexible use of the Interface class through more complex examples

### 1.Multiple Input and Output Components

Suppose you had a more complex function, with multiple inputs and outputs. In the example below, we define a function that takes a string, boolean, and number, and returns a string and number. Take a look how you pass a list of input and output components.

In [35]:
import gradio as gr

def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "checkbox", gr.Slider(0, 100)],
    outputs=["text", "number"],
)
demo.launch()

Running on local URL:  http://127.0.0.1:7882

To create a public link, set `share=True` in `launch()`.


You simply wrap the components in a list. Each component in the inputs list corresponds to one of the parameters of the function, in order. Each component in the outputs list corresponds to one of the values returned by the function, again in order.

### 2.An Image Example

Gradio supports many types of components, such as Image, DataFrame, Video, or Label. Let's try an image-to-image function to get a feel for these!

In [36]:
import numpy as np
import gradio as gr

def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189], 
        [0.349, 0.686, 0.168], 
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

demo = gr.Interface(sepia, gr.Image(shape=(200, 200)), "image")
demo.launch()

Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


When using the Image component as input, your function will receive a NumPy array with the shape (width, height, 3), where the last dimension represents the RGB values. We'll return an image as well in the form of a NumPy array.

You can also set the datatype used by the component with the type= keyword argument. For example, if you wanted your function to take a file path to an image instead of a NumPy array, the input Image component could be written as:


gr.Image(type="filepath", shape=...)
Also note that our input Image component comes with an edit button 🖉, which allows for cropping and zooming into images. Manipulating images in this way can help reveal biases or hidden flaws in a machine learning model!

# Blocks: More Flexibility and Control

Gradio offers two classes to build apps:

1. Interface, that provides a high-level abstraction for creating demos that we've been discussing so far.

2. Blocks, a low-level API for designing web apps with more flexible layouts and data flows. Blocks allows you to do things like feature multiple data flows and demos, control where components appear on the page, handle complex data flows (e.g. outputs can serve as inputs to other functions), and update properties/visibility of components based on user interaction — still all in Python. If this customizability is what you need, try Blocks instead!

### Hello, Blocks

Let's take a look at a simple example. Note how the API here differs from Interface.

In [37]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Textbox(label="Output Box")
    greet_btn = gr.Button("Greet")
    greet_btn.click(fn=greet, inputs=name, outputs=output)

demo.launch()

Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


Things to note:

Blocks are made with a with clause, and any component created inside this clause is automatically added to the app.

Components appear vertically in the app in the order they are created. (Later we will cover customizing layouts!)

A Button was created, and then a click event-listener was added to this button. The API for this should look familiar! Like an Interface, the click method takes a Python function, input components, and output components.

## More Complexity about Blocks

In [38]:
import numpy as np
import gradio as gr

def flip_text(x):
    return x[::-1]

def flip_image(x):
    return np.fliplr(x)

with gr.Blocks() as demo:
    gr.Markdown("Flip text or image files using this demo.")
    with gr.Tab("Flip Text"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
    with gr.Tab("Flip Image"):
        with gr.Row():
            image_input = gr.Image()
            image_output = gr.Image()
        image_button = gr.Button("Flip")

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    text_button.click(flip_text, inputs=text_input, outputs=text_output)
    image_button.click(flip_image, inputs=image_input, outputs=image_output)

demo.launch()

Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


### This tutorial is a simple introductory tutorial for Gradio. For more complex functions, please refer to the URL https://gradio.app/